In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras import layers, models

### Image pre processor for feeding into model

In [2]:
def preprocessImage(imgPath, shape):
    img = cv2.imread(imgPath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (shape))
    img = (img/255).astype(np.float32)
    img = img.T
    img = np.expand_dims(img, axis=-1)
    return img

### For encoding labels into numbers and decoding

In [4]:
label2char ={0: ' ',
 1: "'",
 2: '-',
 3: 'A',
 4: 'B',
 5: 'C',
 6: 'D',
 7: 'E',
 8: 'F',
 9: 'G',
 10: 'H',
 11: 'I',
 12: 'J',
 13: 'K',
 14: 'L',
 15: 'M',
 16: 'N',
 17: 'O',
 18: 'P',
 19: 'Q',
 20: 'R',
 21: 'S',
 22: 'T',
 23: 'U',
 24: 'V',
 25: 'W',
 26: 'X',
 27: 'Y',
 28: 'Z',
 29: '`'}

def getStringFromEncode(lst :list):
    return ''.join([label2char[i] if i in label2char else '' for i in lst])

## Model

In [5]:
model = tf.keras.models.load_model('handwriting_model.h5')

## Prediction

In [6]:
data_path = "/kaggle/input/handwriting-recognition/test_v2/test"
data = pd.read_csv("written_name_test_v2.csv")
data

,FILENAME,IDENTITY
0,TEST_0001.jpg,KEVIN
1,TEST_0002.jpg,CLOTAIRE
2,TEST_0003.jpg,LENA
3,TEST_0004.jpg,JULES
4,TEST_0005.jpg,CHERPIN
...,...,...
41365,TEST_41366.jpg,ALEXANE
41366,TEST_41367.jpg,PEREIRA-SILVA
41367,TEST_41368.jpg,LAURENT
41368,TEST_41369.jpg,DEFFENSE


In [7]:
def decode_batch_predictions(pred):
    pred = pred[:, :-2] # first two layers of ctc garbage
    input_len = np.ones(pred.shape[0])*pred.shape[1]
    
    results = keras.backend.ctc_decode(pred, 
                                        input_length=input_len,
                                        greedy=True)[0][0]
    
    output_text = []
    for res in results.numpy():
        outstr = getStringFromEncode(res)
        output_text.append(outstr)
    
    # return final text results
    return output_text

In [16]:
# reading image and label
index = 10
imgPath = os.path.join(data_path, data["FILENAME"][index])
label = data["IDENTITY"][index]

# showing the image
img = preprocessImage(imgPath, (256,64))
plt.imshow(img)
plt.title(label)

#predicing 
img = np.expand_dims(img, axis=0)
img.shape

preds = model.predict(img)
pred_texts = decode_batch_predictions(preds)
pred_texts


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
